In [261]:
#%pip install pycaret
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection._split import _BaseKFold
import pycaret
import pycaret.regression as pycr 
from sklearn.linear_model import LinearRegression
from pycaret.time_series import TSForecastingExperiment
from statsmodels.tsa.seasonal import seasonal_decompose
import pycaret.utils as pycu

In [262]:
x_train_a = pd.read_csv('cleaned_data/A/x_train_a.csv')
y_train_a = pd.read_csv('cleaned_data/A/train_a.csv')
x_test_a = pd.read_csv('cleaned_data/A/x_test_a.csv')

x_train_b = pd.read_csv('cleaned_data/B/x_train_b.csv')
y_train_b = pd.read_csv('cleaned_data/B/train_b.csv')
x_test_b = pd.read_csv('cleaned_data/B/x_test_b.csv')

x_train_c = pd.read_csv('cleaned_data/C/x_train_c.csv')
y_train_c = pd.read_csv('cleaned_data/C/train_c.csv')
x_test_c = pd.read_csv('cleaned_data/C/x_test_c.csv')

In [ ]:
class CustomTimeSeriesSplit(_BaseKFold):
    def __init__(self, n_splits, train_size=None, test_size=None):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.train_size = train_size
        self.test_size = test_size

    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        indices = np.arange(n_samples)

        # Define initial sizes if not provided
        train_size = self.train_size or n_samples // (self.n_splits + 1)
        test_size = self.test_size or n_samples // self.n_splits

        for test_start in range(train_size + test_size, n_samples, test_size):
            train_end = test_start - test_size
            train_start = max(train_end - train_size, 0)
            yield indices[train_start:train_end], indices[train_end:test_start]

# Example usage
tscv = CustomTimeSeriesSplit(n_splits=5, train_size=720, test_size=120)

for train_idx, test_idx in tscv.split(x_train_a):
    print("Train indices:", train_idx)
    print("Test indices:", test_idx)

In [267]:
x_train_a_combined = x_train_a.merge(y_train_a, left_on='date_forecast', right_on='time', how='left')
x_train_a_combined['observed'] = x_train_a_combined['calc_year'].isna().astype(int)
train_data_a = x_train_a_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour'], inplace=True,axis = 1)
x_test_a['observed'] = x_test_a['calc_year'].isna().astype(int)
test_data_a = x_test_a.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour'], axis = 1)

x_train_b_combined = x_train_b.merge(y_train_b, left_on='date_forecast', right_on='time', how='left')
x_train_b_combined['observed'] = x_train_b_combined['calc_year'].isna().astype(int)
train_data_b = x_train_b_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour'], inplace=True,axis = 1)
x_test_b['observed'] = x_test_b['calc_year'].isna().astype(int)
test_data_b = x_test_b.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour'], axis = 1)

x_train_c_combined = x_train_c.merge(y_train_c, left_on='date_forecast', right_on='time', how='left')
x_train_c_combined['observed'] = x_train_c_combined['calc_year'].isna().astype(int)
train_data_c = x_train_c_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour'],inplace=True, axis = 1)
x_test_c['observed'] = x_test_c['calc_year'].isna().astype(int)
test_data_c = x_test_c.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour'], axis = 1)

In [268]:
#################### A ####################


In [279]:
copyA = x_train_a_combined.copy()
copyA['date_forecast'] = pd.to_datetime(copyA['date_forecast'])
copyA.set_index('date_forecast', inplace=True)

result_yr_seasonal_A = seasonal_decompose(copyA["pv_measurement"], model='additive', period=24*365)
#result_yr_seasonal.plot()
#plt.show()
seasonal_year_A = result_yr_seasonal_A.seasonal
trend_year_A = result_yr_seasonal_A.trend
residual_year_A = result_yr_seasonal_A.resid



df_seasonal_A = pd.DataFrame(index=copyA.index)
df_seasonal_A["seasonal_yr"] = seasonal_year_A

#merging it into training set

x_train_combined_a_seasonal_only_feat = copyA.merge(df_seasonal_A, left_index=True,right_index=True )

x_train_combined_a_seasonal_feat = x_train_combined_a_seasonal_only_feat

column = "seasonal_yr"
x_train_combined_a_seasonal_feat[column] = (x_train_combined_a_seasonal_feat[column] /x_train_combined_a_seasonal_feat[column].abs().max())
#display(x_train_combined_a_seasonal_feat)
#456.062

print(x_train_combined_a_seasonal_feat.loc["2020-07-07 21:00:00","pv_measurement"])
print(x_train_combined_a_seasonal_feat.loc["2020-06-06 20:00:00","pv_measurement"])
x_train_combined_a_seasonal_feat

date_forecast
2019-06-02 22:00:00    2293.098314
2019-06-02 23:00:00    2869.718314
2019-06-03 00:00:00    2624.638314
2019-06-03 01:00:00    2562.964981
2019-06-03 02:00:00    2354.478314
                          ...     
2023-04-30 19:00:00     764.204761
2023-04-30 20:00:00    1357.650119
2023-04-30 21:00:00    1137.317373
2023-04-30 22:00:00     971.162503
2023-04-30 23:00:00     704.352861
Name: seasonal, Length: 34022, dtype: float64

0.22
41.36


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed,seasonal_yr
date_forecast,,,,,,,,,,,,,,,,,,,,,
2019-06-02 22:00:00,7.700,1.22825,1728.950,0.00,0.000,1728.950,0.0,280.300,0.000,0.0,...,-3.575,-0.500,0.0,2019,6,2,22,0.00,1,0.599342
2019-06-02 23:00:00,7.700,1.22350,1689.825,0.00,0.000,1689.825,0.0,280.300,0.000,0.0,...,-3.350,0.275,0.0,2019,6,2,23,0.00,1,0.750052
2019-06-03 00:00:00,7.875,1.21975,1563.225,0.00,0.000,1563.225,0.0,280.650,0.000,0.0,...,-2.950,0.750,0.0,2019,6,3,0,0.00,1,0.685996
2019-06-03 01:00:00,8.425,1.21800,1283.425,834.60,0.750,1283.425,0.0,281.675,0.300,2107.1,...,-2.600,0.875,0.0,2019,6,3,1,0.00,1,0.669876
2019-06-03 02:00:00,8.950,1.21800,1003.500,129872.60,23.100,1003.500,0.0,282.500,11.975,88275.8,...,-2.350,0.925,0.0,2019,6,3,2,19.36,1,0.615384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,4.550,1.27650,1674.200,337859.78,4.225,542.700,0.0,272.425,2.825,225724.2,...,4.800,1.925,0.0,2023,4,30,19,9.02,0,0.199738
2023-04-30 20:00:00,4.500,1.27975,1762.400,9083.50,0.000,546.400,0.0,272.300,0.000,20271.5,...,4.025,2.300,0.0,2023,4,30,20,0.00,0,0.354846
2023-04-30 21:00:00,4.500,1.28100,1696.650,0.00,0.000,548.350,0.0,272.300,0.000,0.0,...,3.575,2.600,0.0,2023,4,30,21,0.00,0,0.297258


In [270]:
def add_lag_feature_and_dropna(dataframe, column_name, lag_hours):
    '''
    Add a lag feature for a given column in a DataFrame with a DateTime index and drop rows with NaN in the lagged column.
    
    Parameters:
    - dataframe: The input DataFrame with a DateTime index.
    - column_name: The name of the column to create a lag feature for.
    - lag_hours: The number of hours to lag.
    
    Returns:
    - A DataFrame with the new lagged feature column added and NaN rows dropped.
    '''
    lag_column_name = f"{column_name}_lag_{lag_hours}"
    dataframe[lag_column_name] = dataframe[column_name].shift(lag_hours)
    dataframe.dropna(subset=[lag_column_name], inplace=True)
    return dataframe

In [271]:
#make seasonal lag feature

dataframe_seasonal_lag_A = add_lag_feature_and_dropna(x_train_combined_a_seasonal_feat, "seasonal_yr", 24*365)
display(dataframe_seasonal_lag_A)
dataframe_seasonal_lag_A.drop(columns= ["seasonal_yr"], inplace= True)
display(dataframe_seasonal_lag_A)

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed,seasonal_yr,seasonal_yr_lag_8760
date_forecast,,,,,,,,,,,,,,,,,,,,,
2020-06-02 08:00:00,7.500,1.22200,6152.650,8987848.00,675.850,2254.150,0.0,280.15002,115.475,1704945.1,...,-1.150,0.0,2020,6,2,8,3751.22,1,0.599342,0.599342
2020-06-02 09:00:00,7.600,1.21650,6818.350,10353380.00,757.225,1702.050,0.0,280.42500,148.800,1902941.8,...,-1.525,0.0,2020,6,2,9,5196.18,1,0.750052,0.750052
2020-06-02 10:00:00,7.825,1.21375,6533.900,11300896.00,806.650,1637.550,0.0,280.90000,188.225,2426745.0,...,-1.800,0.0,2020,6,2,10,5038.00,1,0.685996,0.685996
2020-06-02 11:00:00,8.050,1.21250,6493.475,11759154.00,820.475,1633.225,0.0,281.30000,211.300,2876447.0,...,-1.900,0.0,2020,6,2,11,4487.78,1,0.669876,0.669876
2020-06-02 12:00:00,8.250,1.21225,6660.150,11694132.00,797.700,1516.050,0.0,281.67500,297.175,3660937.5,...,-2.050,0.0,2020,6,2,12,4330.70,1,0.615384,0.615384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,4.550,1.27650,1674.200,337859.78,4.225,542.700,0.0,272.42500,2.825,225724.2,...,1.925,0.0,2023,4,30,19,9.02,0,0.199738,0.199738
2023-04-30 20:00:00,4.500,1.27975,1762.400,9083.50,0.000,546.400,0.0,272.30000,0.000,20271.5,...,2.300,0.0,2023,4,30,20,0.00,0,0.354846,0.354846
2023-04-30 21:00:00,4.500,1.28100,1696.650,0.00,0.000,548.350,0.0,272.30000,0.000,0.0,...,2.600,0.0,2023,4,30,21,0.00,0,0.297258,0.297258


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed,seasonal_yr_lag_8760
date_forecast,,,,,,,,,,,,,,,,,,,,,
2020-06-02 08:00:00,7.500,1.22200,6152.650,8987848.00,675.850,2254.150,0.0,280.15002,115.475,1704945.1,...,1.750,-1.150,0.0,2020,6,2,8,3751.22,1,0.599342
2020-06-02 09:00:00,7.600,1.21650,6818.350,10353380.00,757.225,1702.050,0.0,280.42500,148.800,1902941.8,...,1.650,-1.525,0.0,2020,6,2,9,5196.18,1,0.750052
2020-06-02 10:00:00,7.825,1.21375,6533.900,11300896.00,806.650,1637.550,0.0,280.90000,188.225,2426745.0,...,2.000,-1.800,0.0,2020,6,2,10,5038.00,1,0.685996
2020-06-02 11:00:00,8.050,1.21250,6493.475,11759154.00,820.475,1633.225,0.0,281.30000,211.300,2876447.0,...,2.350,-1.900,0.0,2020,6,2,11,4487.78,1,0.669876
2020-06-02 12:00:00,8.250,1.21225,6660.150,11694132.00,797.700,1516.050,0.0,281.67500,297.175,3660937.5,...,2.750,-2.050,0.0,2020,6,2,12,4330.70,1,0.615384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,4.550,1.27650,1674.200,337859.78,4.225,542.700,0.0,272.42500,2.825,225724.2,...,4.800,1.925,0.0,2023,4,30,19,9.02,0,0.199738
2023-04-30 20:00:00,4.500,1.27975,1762.400,9083.50,0.000,546.400,0.0,272.30000,0.000,20271.5,...,4.025,2.300,0.0,2023,4,30,20,0.00,0,0.354846
2023-04-30 21:00:00,4.500,1.28100,1696.650,0.00,0.000,548.350,0.0,272.30000,0.000,0.0,...,3.575,2.600,0.0,2023,4,30,21,0.00,0,0.297258


In [272]:
display(test_data_a_seasonal_lag)
display(dataframe_seasonal_lag_A)
df1_columns = set(test_data_a_seasonal_lag.columns)
df2_columns = set(dataframe_seasonal_lag_A.columns)

if df1_columns == df2_columns:
    print("The columns of both dataframes match exactly!")
else:
    missing_in_df1 = df2_columns - df1_columns
    missing_in_df2 = df1_columns - df2_columns
    
    if missing_in_df1:
        print("Columns present in dataframe2 but missing in dataframe1:", missing_in_df1)
    if missing_in_df2:
        print("Columns present in dataframe1 but missing in dataframe2:", missing_in_df2)

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,observed,seasonal_yr_lag_8760
2023-05-01 00:00:00,4.325,1.28675,912.7000,0.00,0.000,1061.5500,0.0,271.65002,0.000,0.00,...,3.950,2.100,3.350,0.0,2023,5,1,0,0,-0.068598
2023-05-01 01:00:00,4.275,1.28600,1482.1000,0.00,0.000,1075.1001,0.0,271.45000,0.000,0.00,...,3.825,1.925,3.300,0.0,2023,5,1,1,0,0.000445
2023-05-01 02:00:00,4.150,1.28375,1791.3000,0.00,0.000,1200.4000,0.0,271.05000,0.000,0.00,...,3.650,1.750,3.200,0.0,2023,5,1,2,0,0.138531
2023-05-01 03:00:00,4.025,1.28200,2312.8750,40497.70,11.675,1179.8500,0.0,270.65000,9.375,67380.91,...,3.500,1.450,3.150,0.0,2023,5,1,3,0,0.357174
2023-05-01 04:00:00,3.900,1.28100,2198.2998,566994.40,76.875,920.0500,0.0,270.37500,47.400,408838.80,...,3.325,1.300,3.050,0.0,2023,5,1,4,0,0.552807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-03 19:00:00,8.350,1.19725,3638.9000,1908372.80,85.100,2013.7500,0.0,281.57500,33.625,675098.20,...,2.450,2.075,-1.350,0.0,2023,7,3,19,0,0.373247
2023-07-03 20:00:00,8.525,1.20050,3552.1000,737342.70,24.800,1610.9000,0.0,281.85000,14.325,345284.50,...,2.450,2.100,-1.250,0.0,2023,7,3,20,0,0.378999
2023-07-03 21:00:00,8.825,1.20450,2315.0000,149717.31,1.275,1622.8000,0.0,282.32500,1.300,112676.60,...,2.575,2.150,-1.400,0.0,2023,7,3,21,0,0.367490
2023-07-03 22:00:00,9.025,1.20700,2202.8000,1440.20,0.000,1767.5500,0.0,282.67502,0.000,9402.90,...,2.250,1.800,-1.350,0.0,2023,7,3,22,0,0.332963


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed,seasonal_yr_lag_8760
date_forecast,,,,,,,,,,,,,,,,,,,,,
2020-06-02 08:00:00,7.500,1.22200,6152.650,8987848.00,675.850,2254.150,0.0,280.15002,115.475,1704945.1,...,1.750,-1.150,0.0,2020,6,2,8,3751.22,1,0.599342
2020-06-02 09:00:00,7.600,1.21650,6818.350,10353380.00,757.225,1702.050,0.0,280.42500,148.800,1902941.8,...,1.650,-1.525,0.0,2020,6,2,9,5196.18,1,0.750052
2020-06-02 10:00:00,7.825,1.21375,6533.900,11300896.00,806.650,1637.550,0.0,280.90000,188.225,2426745.0,...,2.000,-1.800,0.0,2020,6,2,10,5038.00,1,0.685996
2020-06-02 11:00:00,8.050,1.21250,6493.475,11759154.00,820.475,1633.225,0.0,281.30000,211.300,2876447.0,...,2.350,-1.900,0.0,2020,6,2,11,4487.78,1,0.669876
2020-06-02 12:00:00,8.250,1.21225,6660.150,11694132.00,797.700,1516.050,0.0,281.67500,297.175,3660937.5,...,2.750,-2.050,0.0,2020,6,2,12,4330.70,1,0.615384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,4.550,1.27650,1674.200,337859.78,4.225,542.700,0.0,272.42500,2.825,225724.2,...,4.800,1.925,0.0,2023,4,30,19,9.02,0,0.199738
2023-04-30 20:00:00,4.500,1.27975,1762.400,9083.50,0.000,546.400,0.0,272.30000,0.000,20271.5,...,4.025,2.300,0.0,2023,4,30,20,0.00,0,0.354846
2023-04-30 21:00:00,4.500,1.28100,1696.650,0.00,0.000,548.350,0.0,272.30000,0.000,0.0,...,3.575,2.600,0.0,2023,4,30,21,0.00,0,0.297258


Columns present in dataframe2 but missing in dataframe1: {'pv_measurement'}


In [273]:
#pycaret_ml_A = setup(data = dataframe_seasonal_lag_A, target = "pv_measurement")
pycaret_ml_A = setup(data = dataframe_seasonal_lag_A, target = "pv_measurement")


,Description,Value
0,Session id,4028
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(25262, 52)"
4,Transformed data shape,"(25262, 52)"
5,Transformed train set shape,"(17683, 52)"
6,Transformed test set shape,"(7579, 52)"
7,Numeric features,51
8,Rows with missing values,96.7%
9,Preprocess,True


In [274]:
best_model_A = compare_models(sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,135.6787,90894.5623,301.0369,0.9352,1.1616,1.3250,0.3100
catboost,CatBoost Regressor,138.2740,86344.3576,293.4477,0.9385,1.8614,2.2314,10.0070
et,Extra Trees Regressor,139.1877,101774.3846,318.7300,0.9274,0.5356,1.1488,3.9380
rf,Random Forest Regressor,141.2678,106580.2378,326.1222,0.9241,0.5450,1.1943,14.0000
xgboost,Extreme Gradient Boosting,147.1834,105346.0906,324.3559,0.9249,1.5637,1.7138,3.5220
gbr,Gradient Boosting Regressor,180.3320,136313.6618,368.9343,0.9028,1.9528,2.7738,5.1990
dt,Decision Tree Regressor,194.1917,221911.8417,470.4090,0.8418,0.6779,1.3218,0.3520
huber,Huber Regressor,243.5059,275539.3920,524.5148,0.8038,0.9508,2.7099,0.2860
en,Elastic Net,245.5469,216334.6414,464.7402,0.8460,2.7954,4.7408,0.5250
llar,Lasso Least Angle Regression,255.6046,206751.5693,454.3587,0.8527,3.0002,7.2416,0.0500


In [115]:
#################### B ####################


In [296]:
copyB = x_train_b_combined.copy()
copyB['date_forecast'] = pd.to_datetime(copyB['date_forecast'])
copyB.set_index('date_forecast', inplace=True)

result_yr_seasonal_B = seasonal_decompose(copyB["pv_measurement"], model='additive', period=24*365)
#result_yr_seasonal.plot()
#plt.show()
seasonal_year_B = result_yr_seasonal_B.seasonal
trend_year_B = result_yr_seasonal_B.trend
residual_year_B = result_yr_seasonal_B.resid




df_seasonal_B = pd.DataFrame(index=copyB.index)
df_seasonal_B["seasonal_yr"] = seasonal_year_B
#merging it into training set

x_train_combined_b_seasonal_only_feat = copyB.merge(df_seasonal_B, left_index=True,right_index=True )
x_train_combined_b_seasonal_feat = x_train_combined_b_seasonal_only_feat
column = "seasonal_yr"
x_train_combined_b_seasonal_feat[column] = x_train_combined_b_seasonal_feat[column] /x_train_combined_b_seasonal_feat[column].abs().max()
display(x_train_combined_b_seasonal_feat)


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed,seasonal_yr
date_forecast,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,5.525,1.23975,1200.6750,0.0,0.000,1200.6750,0.0,275.150,0.000,0.00,...,1.175,2.550,0.0,2019,1,1,0,0.000000,1,-0.124526
2019-01-01 01:00:00,5.425,1.23975,1131.4249,0.0,0.000,1131.4249,0.0,274.825,0.000,0.00,...,1.525,3.200,0.0,2019,1,1,1,0.000000,1,-0.106586
2019-01-01 02:00:00,5.400,1.23850,1061.0000,0.0,0.000,1061.0000,0.0,274.800,0.000,0.00,...,2.150,3.425,0.0,2019,1,1,2,0.000000,1,-0.103324
2019-01-01 03:00:00,5.350,1.23975,1021.1500,0.0,0.000,1021.1500,0.0,274.675,0.000,0.00,...,3.425,3.500,0.0,2019,1,1,3,0.000000,1,-0.161220
2019-01-01 04:00:00,5.675,1.23750,1033.7000,0.0,0.000,1033.7000,0.0,275.500,0.000,0.00,...,5.575,2.200,0.0,2019,1,1,4,0.000000,1,-0.181605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,4.550,1.27650,1677.9500,337850.1,4.225,542.8500,0.0,272.425,2.825,225735.89,...,4.800,1.925,0.0,2023,4,30,19,0.828587,0,0.065367
2023-04-30 20:00:00,4.500,1.27875,1766.5000,9083.1,0.000,546.3500,0.0,272.300,0.000,20268.10,...,4.025,2.300,0.0,2023,4,30,20,-0.000000,0,0.058840
2023-04-30 21:00:00,4.500,1.27900,1698.9250,0.0,0.000,548.0500,0.0,272.300,0.000,0.00,...,3.600,2.600,0.0,2023,4,30,21,-0.000000,0,0.098795


In [282]:
#make seasonal lag feature

dataframe_seasonal_lag_B = add_lag_feature_and_dropna(x_train_combined_b_seasonal_feat, "seasonal_yr", 24*365)
dataframe_seasonal_lag_B.drop(columns= ["seasonal_yr"], inplace= True)
display(dataframe_seasonal_lag_B)

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed,seasonal_yr_lag_8760
date_forecast,,,,,,,,,,,,,,,,,,,,,
2020-02-03 11:00:00,4.175,1.27625,1123.9250,1579067.5,116.675,1123.925,0.0,271.15000,48.824997,657983.60,...,-0.250,1.500,0.0,2020,2,3,11,50.025000,1,-0.124526
2020-02-03 12:00:00,4.225,1.27350,1185.0250,1637795.4,105.850,1185.025,0.0,271.25000,60.650000,788205.70,...,-0.525,1.225,0.0,2020,2,3,12,79.350000,1,-0.106586
2020-02-03 13:00:00,4.100,1.27300,1139.9500,1286438.6,69.475,1139.950,0.0,270.90000,46.750000,773376.20,...,-0.875,0.950,0.0,2020,2,3,13,82.800000,1,-0.103324
2020-02-03 14:00:00,4.100,1.27325,1094.8250,673664.8,25.000,1094.825,0.0,270.84998,22.450000,498368.70,...,-1.750,0.550,0.0,2020,2,3,14,21.562500,1,-0.161220
2020-02-03 15:00:00,3.975,1.27525,1108.0500,136991.6,0.350,1108.050,0.0,270.50000,0.425000,164803.10,...,-2.700,0.750,0.0,2020,2,3,15,0.000000,1,-0.181605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,4.550,1.27650,1677.9500,337850.1,4.225,542.850,0.0,272.42500,2.825000,225735.89,...,4.800,1.925,0.0,2023,4,30,19,0.828587,0,0.065367
2023-04-30 20:00:00,4.500,1.27875,1766.5000,9083.1,0.000,546.350,0.0,272.30000,0.000000,20268.10,...,4.025,2.300,0.0,2023,4,30,20,-0.000000,0,0.058840
2023-04-30 21:00:00,4.500,1.27900,1698.9250,0.0,0.000,548.050,0.0,272.30000,0.000000,0.00,...,3.600,2.600,0.0,2023,4,30,21,-0.000000,0,0.098795


In [283]:
pycaret_ml_B = setup(data = dataframe_seasonal_lag_B, target = "pv_measurement")

,Description,Value
0,Session id,2540
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(18293, 52)"
4,Transformed data shape,"(18293, 52)"
5,Transformed train set shape,"(12805, 52)"
6,Transformed test set shape,"(5488, 52)"
7,Numeric features,51
8,Rows with missing values,97.2%
9,Preprocess,True


In [284]:
best_model_B = compare_models(sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,13.5539,1140.0747,33.7092,0.9721,0.6746,1.2626,2.6650
lightgbm,Light Gradient Boosting Machine,13.7048,1064.7918,32.5681,0.9740,0.7907,1.1331,0.2650
catboost,CatBoost Regressor,14.1043,995.9877,31.5044,0.9757,0.9908,1.2108,8.8580
rf,Random Forest Regressor,14.5710,1332.5409,36.4475,0.9674,0.7134,1.2117,10.7320
xgboost,Extreme Gradient Boosting,14.9073,1184.4565,34.3595,0.9710,0.9305,1.1365,2.5790
dt,Decision Tree Regressor,19.0441,2708.5759,51.8925,0.9340,0.7506,1.1205,0.2120
gbr,Gradient Boosting Regressor,20.5858,1955.0255,44.1134,0.9523,1.1943,1.8054,3.7090
huber,Huber Regressor,39.0133,7402.3202,85.9215,0.8196,0.9481,3.2258,0.2070
en,Elastic Net,39.3534,5867.5237,76.4543,0.8570,1.9115,3.5584,0.3330
knn,K Neighbors Regressor,39.8674,7903.7425,88.7848,0.8073,1.0030,3.0041,0.0890


In [ ]:
#################### C ####################


In [299]:
copyC = x_train_c_combined.copy()
copyC['date_forecast'] = pd.to_datetime(copyC['date_forecast'])
copyC.set_index('date_forecast', inplace=True)

result_yr_seasonal_C = seasonal_decompose(copyC["pv_measurement"], model='additive', period=24*365)
#result_yr_seasonal.plot()
#plt.show()
seasonal_year_C = result_yr_seasonal_C.seasonal
trend_year_C = result_yr_seasonal_C.trend
residual_year_C = result_yr_seasonal_C.resid



df_seasonal_C = pd.DataFrame(index=copyC.index)
df_seasonal_C["seasonal_yr"] = seasonal_year_C
#merging it into training set

x_train_combined_c_seasonal_only_feat = copyC.merge(df_seasonal_C, left_index=True,right_index=True )
x_train_combined_c_seasonal_feat = x_train_combined_c_seasonal_only_feat
column = "seasonal_yr"
x_train_combined_c_seasonal_feat[column] = x_train_combined_c_seasonal_feat[column] /x_train_combined_c_seasonal_feat[column].abs().max()
display(x_train_combined_c_seasonal_feat)
#456.062
print(x_train_combined_c_seasonal_feat.loc["2020-07-07 21:00:00","pv_measurement"])
print(x_train_combined_c_seasonal_feat.loc["2020-06-06 20:00:00","pv_measurement"])


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed,seasonal_yr
date_forecast,,,,,,,,,,,,,,,,,,,,,
2019-09-04 08:00:00,6.625,1.22075,2287.250,5283377.5,421.225,2287.250,0.0,278.20000,67.975,932039.7,...,-0.825,0.825,0.0,2019,9,4,8,137.20,1,-0.153865
2019-09-04 09:00:00,6.275,1.21425,2679.075,6726922.0,508.125,2679.075,0.0,277.42500,76.625,1041404.3,...,-1.500,-0.200,0.0,2019,9,4,9,0.00,1,-0.153851
2019-09-04 10:00:00,5.900,1.20825,2983.750,7747199.5,561.875,2983.750,0.0,276.65002,112.575,1362371.1,...,-2.000,-0.225,0.0,2019,9,4,10,0.00,1,-0.153834
2019-09-04 11:00:00,5.875,1.20350,3286.550,8254105.0,578.025,3286.550,0.0,276.65000,195.150,2215590.5,...,-1.950,-0.250,0.0,2019,9,4,11,0.00,1,-0.153813
2019-09-04 12:00:00,6.150,1.20175,3453.425,8205280.0,555.175,3453.425,0.0,277.25000,243.375,3157120.2,...,-1.500,-0.300,0.0,2019,9,4,12,0.00,1,-0.153789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,4.400,1.27550,1456.575,336040.6,4.175,551.225,0.0,272.02500,2.775,219096.0,...,3.600,1.875,0.0,2023,4,30,19,50.96,0,0.621915
2023-04-30 20:00:00,4.400,1.27850,1476.350,8827.2,0.000,564.100,0.0,271.95000,0.000,19936.2,...,2.950,2.125,0.0,2023,4,30,20,2.94,0,0.368760
2023-04-30 21:00:00,4.400,1.27900,1516.300,0.0,0.000,578.700,0.0,271.90000,0.000,0.0,...,2.625,2.400,0.0,2023,4,30,21,0.00,0,0.437803


0.0
9.8


In [286]:
#make seasonal lag feature

dataframe_seasonal_lag_C = add_lag_feature_and_dropna(x_train_combined_c_seasonal_feat, "seasonal_yr", 24*365)
dataframe_seasonal_lag_C.drop(columns= ["seasonal_yr"], inplace= True)
display(dataframe_seasonal_lag_C)

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed,seasonal_yr_lag_8760
date_forecast,,,,,,,,,,,,,,,,,,,,,
2020-10-24 20:00:00,4.050,1.27000,5255.675,0.0,0.000,4219.625,-1.0,270.775,0.000,0.0,...,-1.800,0.700,0.0,2020,10,24,20,0.00,1,-0.153865
2020-10-24 21:00:00,4.250,1.26150,3885.900,0.0,0.000,3425.600,0.0,271.400,0.000,0.0,...,-1.800,0.625,0.0,2020,10,24,21,0.00,1,-0.153851
2020-10-24 22:00:00,4.475,1.25300,3195.300,0.0,0.000,2897.650,0.0,272.100,0.000,0.0,...,-2.200,0.500,0.0,2020,10,24,22,0.00,1,-0.153834
2020-10-24 23:00:00,4.725,1.24675,3323.475,0.0,0.000,3279.525,0.0,273.000,0.000,0.0,...,-2.300,0.650,0.0,2020,10,24,23,0.00,1,-0.153813
2020-10-25 02:00:00,5.250,1.23800,4204.350,0.0,0.000,2803.450,0.0,274.500,0.000,0.0,...,-2.350,1.275,0.0,2020,10,25,2,0.00,1,-0.153789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,4.400,1.27550,1456.575,336040.6,4.175,551.225,0.0,272.025,2.775,219096.0,...,3.600,1.875,0.0,2023,4,30,19,50.96,0,0.621915
2023-04-30 20:00:00,4.400,1.27850,1476.350,8827.2,0.000,564.100,0.0,271.950,0.000,19936.2,...,2.950,2.125,0.0,2023,4,30,20,2.94,0,0.368760
2023-04-30 21:00:00,4.400,1.27900,1516.300,0.0,0.000,578.700,0.0,271.900,0.000,0.0,...,2.625,2.400,0.0,2023,4,30,21,0.00,0,0.437803


In [287]:
pycaret_ml_C = setup(data = dataframe_seasonal_lag_C, target = "pv_measurement")

,Description,Value
0,Session id,1538
1,Target,pv_measurement
2,Target type,Regression
3,Original data shape,"(12924, 52)"
4,Transformed data shape,"(12924, 52)"
5,Transformed train set shape,"(9046, 52)"
6,Transformed test set shape,"(3878, 52)"
7,Numeric features,51
8,Rows with missing values,99.3%
9,Preprocess,True


In [288]:
best_model_C = compare_models(sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,12.0099,733.5743,27.0235,0.9736,0.7083,0.5427,0.2250
catboost,CatBoost Regressor,12.0831,676.0058,25.9280,0.9756,0.8539,0.4734,7.6860
et,Extra Trees Regressor,12.1916,837.5991,28.8798,0.9698,0.6089,0.5735,1.6000
rf,Random Forest Regressor,12.4059,949.9686,30.7477,0.9658,0.6070,0.6047,6.0660
xgboost,Extreme Gradient Boosting,13.3813,931.8480,30.4007,0.9664,0.7611,0.5188,1.8510
dt,Decision Tree Regressor,16.4175,2096.3016,45.4760,0.9238,0.6943,0.7109,0.1250
gbr,Gradient Boosting Regressor,19.1281,1585.4365,39.6946,0.9431,1.1174,0.7841,2.6150
knn,K Neighbors Regressor,29.6394,4492.8588,66.8934,0.8389,0.9025,1.2028,0.0550
huber,Huber Regressor,29.9988,4610.9950,67.7822,0.8348,0.8850,1.3307,0.1140
omp,Orthogonal Matching Pursuit,33.2525,4459.1851,66.6895,0.8402,1.5845,1.4362,0.0350


In [ ]:
############ Add seasonal lag to test sets ##################

In [289]:
def set_datetime_index_corrected(dataframe, year_col='forecast_year', month_col='forecast_month', day_col='forecast_day', hour_col='forecast_hour'):

    # Combine the year, month, day, and hour columns to create a datetime series
    datetime_series = pd.to_datetime(dataframe[year_col].astype(str) + '-' + 
                                     dataframe[month_col].astype(str) + '-' + 
                                     dataframe[day_col].astype(str) + ' ' + 
                                     dataframe[hour_col].astype(str) + ':00:00')
    
    # Set this datetime series as the dataframe's index
    dataframe.set_index(datetime_series, inplace=True)
    
    # Return the modified dataframe
    return dataframe

In [290]:
##### A ####
#Check shape of both of train with seasonality and test data, find out when test data starts
#date time index
#x_train_combined_a_seasonal_feat
display(test_data_a)

test_data_a_copy = test_data_a.copy().drop(columns=["location"])
x_train_combined_a_seasonal_feat_copy = x_train_combined_a_seasonal_feat.copy()


test_data_a_dtidx = set_datetime_index_corrected(test_data_a_copy,year_col='forecast_year', month_col='forecast_month', day_col='forecast_day', hour_col='forecast_hour')
print(test_data_a_dtidx.shape)

display(x_train_combined_a_seasonal_feat_copy)

combined_test_train_a = pd.concat([x_train_combined_a_seasonal_feat_copy,test_data_a_dtidx], axis =0)

combined_test_train_seas_lag_a = add_lag_feature_and_dropna(combined_test_train_a, "seasonal_yr", 24*365)


test_data_a_seasonal_lag = combined_test_train_seas_lag_a.loc["2023-05-01 00:00:00":]

test_data_a_seasonal_lag.drop(columns=["seasonal_yr","pv_measurement"], inplace= True)
display(test_data_a_seasonal_lag)
#train data with seasonality concat with test data --> nan values for seasonality


#make seasonal lag feature

#cut where test started (check that it has the same shape as before)

,location,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,observed
0,A,4.325,1.28675,912.7000,0.00,0.000,1061.5500,0.0,271.65002,0.000,...,29907.500,3.950,2.100,3.350,0.0,2023,5,1,0,0
1,A,4.275,1.28600,1482.1000,0.00,0.000,1075.1001,0.0,271.45000,0.000,...,29519.074,3.825,1.925,3.300,0.0,2023,5,1,1,0
2,A,4.150,1.28375,1791.3000,0.00,0.000,1200.4000,0.0,271.05000,0.000,...,31009.125,3.650,1.750,3.200,0.0,2023,5,1,2,0
3,A,4.025,1.28200,2312.8750,40497.70,11.675,1179.8500,0.0,270.65000,9.375,...,34552.500,3.500,1.450,3.150,0.0,2023,5,1,3,0
4,A,3.900,1.28100,2198.2998,566994.40,76.875,920.0500,0.0,270.37500,47.400,...,35483.875,3.325,1.300,3.050,0.0,2023,5,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,A,8.350,1.19725,3638.9000,1908372.80,85.100,2013.7500,0.0,281.57500,33.625,...,44056.375,2.450,2.075,-1.350,0.0,2023,7,3,19,0
716,A,8.525,1.20050,3552.1000,737342.70,24.800,1610.9000,0.0,281.85000,14.325,...,44017.176,2.450,2.100,-1.250,0.0,2023,7,3,20,0
717,A,8.825,1.20450,2315.0000,149717.31,1.275,1622.8000,0.0,282.32500,1.300,...,43302.050,2.575,2.150,-1.400,0.0,2023,7,3,21,0
718,A,9.025,1.20700,2202.8000,1440.20,0.000,1767.5500,0.0,282.67502,0.000,...,40505.850,2.250,1.800,-1.350,0.0,2023,7,3,22,0


(720, 50)


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed,seasonal_yr
date_forecast,,,,,,,,,,,,,,,,,,,,,
2019-06-02 22:00:00,7.700,1.22825,1728.950,0.00,0.000,1728.950,0.0,280.300,0.000,0.0,...,-3.575,-0.500,0.0,2019,6,2,22,0.00,1,0.599342
2019-06-02 23:00:00,7.700,1.22350,1689.825,0.00,0.000,1689.825,0.0,280.300,0.000,0.0,...,-3.350,0.275,0.0,2019,6,2,23,0.00,1,0.750052
2019-06-03 00:00:00,7.875,1.21975,1563.225,0.00,0.000,1563.225,0.0,280.650,0.000,0.0,...,-2.950,0.750,0.0,2019,6,3,0,0.00,1,0.685996
2019-06-03 01:00:00,8.425,1.21800,1283.425,834.60,0.750,1283.425,0.0,281.675,0.300,2107.1,...,-2.600,0.875,0.0,2019,6,3,1,0.00,1,0.669876
2019-06-03 02:00:00,8.950,1.21800,1003.500,129872.60,23.100,1003.500,0.0,282.500,11.975,88275.8,...,-2.350,0.925,0.0,2019,6,3,2,19.36,1,0.615384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,4.550,1.27650,1674.200,337859.78,4.225,542.700,0.0,272.425,2.825,225724.2,...,4.800,1.925,0.0,2023,4,30,19,9.02,0,0.199738
2023-04-30 20:00:00,4.500,1.27975,1762.400,9083.50,0.000,546.400,0.0,272.300,0.000,20271.5,...,4.025,2.300,0.0,2023,4,30,20,0.00,0,0.354846
2023-04-30 21:00:00,4.500,1.28100,1696.650,0.00,0.000,548.350,0.0,272.300,0.000,0.0,...,3.575,2.600,0.0,2023,4,30,21,0.00,0,0.297258


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,observed,seasonal_yr_lag_8760
2023-05-01 00:00:00,4.325,1.28675,912.7000,0.00,0.000,1061.5500,0.0,271.65002,0.000,0.00,...,3.950,2.100,3.350,0.0,2023,5,1,0,0,0.124771
2023-05-01 01:00:00,4.275,1.28600,1482.1000,0.00,0.000,1075.1001,0.0,271.45000,0.000,0.00,...,3.825,1.925,3.300,0.0,2023,5,1,1,0,0.047417
2023-05-01 02:00:00,4.150,1.28375,1791.3000,0.00,0.000,1200.4000,0.0,271.05000,0.000,0.00,...,3.650,1.750,3.200,0.0,2023,5,1,2,0,-0.040966
2023-05-01 03:00:00,4.025,1.28200,2312.8750,40497.70,11.675,1179.8500,0.0,270.65000,9.375,67380.91,...,3.500,1.450,3.150,0.0,2023,5,1,3,0,-0.112342
2023-05-01 04:00:00,3.900,1.28100,2198.2998,566994.40,76.875,920.0500,0.0,270.37500,47.400,408838.80,...,3.325,1.300,3.050,0.0,2023,5,1,4,0,-0.145538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-03 19:00:00,8.350,1.19725,3638.9000,1908372.80,85.100,2013.7500,0.0,281.57500,33.625,675098.20,...,2.450,2.075,-1.350,0.0,2023,7,3,19,0,0.377831
2023-07-03 20:00:00,8.525,1.20050,3552.1000,737342.70,24.800,1610.9000,0.0,281.85000,14.325,345284.50,...,2.450,2.100,-1.250,0.0,2023,7,3,20,0,0.205424
2023-07-03 21:00:00,8.825,1.20450,2315.0000,149717.31,1.275,1622.8000,0.0,282.32500,1.300,112676.60,...,2.575,2.150,-1.400,0.0,2023,7,3,21,0,0.141349
2023-07-03 22:00:00,9.025,1.20700,2202.8000,1440.20,0.000,1767.5500,0.0,282.67502,0.000,9402.90,...,2.250,1.800,-1.350,0.0,2023,7,3,22,0,0.331276


In [297]:
##### B ####
#Check shape of both of train with seasonality and test data, find out when test data starts
#date time index
#x_train_combined_a_seasonal_feat

test_data_b_copy = test_data_b.copy().drop(columns=["location"])
x_train_combined_b_seasonal_feat_copy = x_train_combined_b_seasonal_feat.copy()


test_data_b_dtidx = set_datetime_index_corrected(test_data_b_copy,year_col='forecast_year', month_col='forecast_month', day_col='forecast_day', hour_col='forecast_hour')
print(test_data_b_dtidx.shape)

display(x_train_combined_b_seasonal_feat)

combined_test_train_b = pd.concat([x_train_combined_b_seasonal_feat_copy,test_data_b_dtidx], axis =0)



combined_test_train_seas_lag_b = add_lag_feature_and_dropna(combined_test_train_b, "seasonal_yr", 24*365)


test_data_b_seasonal_lag = combined_test_train_seas_lag_b.loc["2023-05-01 00:00:00":]

test_data_b_seasonal_lag.drop(columns=["seasonal_yr","pv_measurement"], inplace= True)
display(test_data_b_seasonal_lag)
#train data with seasonality concat with test data --> nan values for seasonality


#make seasonal lag feature

#cut where test started (check that it has the same shape as before)


(720, 50)


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,pv_measurement,observed,seasonal_yr
date_forecast,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,5.525,1.23975,1200.6750,0.0,0.000,1200.6750,0.0,275.150,0.000,0.00,...,1.175,2.550,0.0,2019,1,1,0,0.000000,1,-0.124526
2019-01-01 01:00:00,5.425,1.23975,1131.4249,0.0,0.000,1131.4249,0.0,274.825,0.000,0.00,...,1.525,3.200,0.0,2019,1,1,1,0.000000,1,-0.106586
2019-01-01 02:00:00,5.400,1.23850,1061.0000,0.0,0.000,1061.0000,0.0,274.800,0.000,0.00,...,2.150,3.425,0.0,2019,1,1,2,0.000000,1,-0.103324
2019-01-01 03:00:00,5.350,1.23975,1021.1500,0.0,0.000,1021.1500,0.0,274.675,0.000,0.00,...,3.425,3.500,0.0,2019,1,1,3,0.000000,1,-0.161220
2019-01-01 04:00:00,5.675,1.23750,1033.7000,0.0,0.000,1033.7000,0.0,275.500,0.000,0.00,...,5.575,2.200,0.0,2019,1,1,4,0.000000,1,-0.181605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-30 19:00:00,4.550,1.27650,1677.9500,337850.1,4.225,542.8500,0.0,272.425,2.825,225735.89,...,4.800,1.925,0.0,2023,4,30,19,0.828587,0,0.065367
2023-04-30 20:00:00,4.500,1.27875,1766.5000,9083.1,0.000,546.3500,0.0,272.300,0.000,20268.10,...,4.025,2.300,0.0,2023,4,30,20,-0.000000,0,0.058840
2023-04-30 21:00:00,4.500,1.27900,1698.9250,0.0,0.000,548.0500,0.0,272.300,0.000,0.00,...,3.600,2.600,0.0,2023,4,30,21,-0.000000,0,0.098795


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,observed,seasonal_yr_lag_8760
2023-05-01 00:00:00,4.300,1.28300,912.3000,0.0,0.000,1059.750,0.0,271.65002,0.000,0.000,...,3.950,2.100,3.375,0.0,2023,5,1,0,0,0.273297
2023-05-01 01:00:00,4.250,1.28300,1482.8002,0.0,0.000,1073.700,0.0,271.45000,0.000,0.000,...,3.825,1.925,3.300,0.0,2023,5,1,1,0,0.340977
2023-05-01 02:00:00,4.150,1.28275,1765.9000,0.0,0.000,1200.100,0.0,271.05000,0.000,0.000,...,3.650,1.750,3.225,0.0,2023,5,1,2,0,0.764185
2023-05-01 03:00:00,4.025,1.28225,2269.7500,40510.2,11.675,1179.000,0.0,270.65000,9.375,67382.305,...,3.500,1.475,3.150,0.0,2023,5,1,3,0,0.650029
2023-05-01 04:00:00,3.900,1.28200,2198.2250,567057.1,76.900,919.150,0.0,270.37500,47.400,408812.200,...,3.325,1.300,3.075,0.0,2023,5,1,4,0,0.305933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-03 19:00:00,8.350,1.19800,3640.1250,1908360.9,85.100,2015.750,0.0,281.57500,33.625,675011.000,...,2.475,2.075,-1.350,0.0,2023,7,3,19,0,-0.155935
2023-07-03 20:00:00,8.525,1.20075,3351.1000,737351.8,24.800,1613.375,0.0,281.85000,14.350,345239.800,...,2.450,2.100,-1.275,0.0,2023,7,3,20,0,-0.138019
2023-07-03 21:00:00,8.800,1.20375,2753.0250,149728.8,1.275,1624.450,0.0,282.30000,1.300,112669.700,...,2.575,2.150,-1.400,0.0,2023,7,3,21,0,-0.096462
2023-07-03 22:00:00,9.000,1.20600,2204.5000,1440.5,0.000,1768.325,0.0,282.67502,0.000,9413.900,...,2.250,1.800,-1.350,0.0,2023,7,3,22,0,-0.025552


In [300]:
##### C ####
#Check shape of both of train with seasonality and test data, find out when test data starts
#date time index
#x_train_combined_a_seasonal_feat
display(test_data_c)

test_data_c_copy = test_data_c.copy().drop(columns=["location"])
x_train_combined_c_seasonal_feat_copy = x_train_combined_c_seasonal_feat.copy()




test_data_c_dtidx = set_datetime_index_corrected(test_data_c_copy,year_col='forecast_year', month_col='forecast_month', day_col='forecast_day', hour_col='forecast_hour')
print(test_data_c_dtidx.shape)


combined_test_train_c = pd.concat([x_train_combined_c_seasonal_feat_copy,test_data_c_dtidx], axis =0)

combined_test_train_seas_lag_c = add_lag_feature_and_dropna(combined_test_train_c, "seasonal_yr", 24*365)


test_data_c_seasonal_lag = combined_test_train_seas_lag_c.loc["2023-05-01 00:00:00":]

test_data_c_seasonal_lag.drop(columns=["seasonal_yr","pv_measurement"], inplace= True)
display(test_data_c_seasonal_lag)
#train data with seasonality concat with test data --> nan values for seasonality


,location,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,observed
0,C,4.150,1.28600,NaN,0.0,0.000000,1236.500,0.0,271.025,0.000,...,31241.975,3.425,1.450,3.100,0.0,2023,5,1,0,0
1,C,4.050,1.28550,1477.100,0.0,0.000000,1220.425,0.0,270.700,0.000,...,31019.550,3.325,1.400,3.025,0.0,2023,5,1,1,0
2,C,3.900,1.28375,1477.100,0.0,0.000000,1258.650,0.0,270.200,0.000,...,32372.800,3.225,1.225,3.000,0.0,2023,5,1,2,0
3,C,3.750,1.28275,NaN,39890.0,11.549999,1655.175,0.0,269.700,9.300,...,35433.625,3.125,0.950,3.000,0.0,2023,5,1,3,0
4,C,3.700,1.28150,1990.000,564426.7,76.675000,1639.100,0.0,269.450,47.925,...,35455.574,3.025,0.700,2.950,0.0,2023,5,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,C,8.400,1.19675,3493.650,1903925.5,84.775000,1967.375,0.0,281.700,32.200,...,41007.900,2.175,1.900,-1.075,0.0,2023,7,3,19,0
716,C,8.600,1.20000,3078.750,733045.4,24.550000,1449.500,0.0,282.025,13.875,...,41315.950,2.200,2.000,-0.925,0.0,2023,7,3,20,0
717,C,8.875,1.20350,2308.400,147324.3,1.225000,1543.650,0.0,282.350,1.250,...,41665.900,2.250,2.050,-0.950,0.0,2023,7,3,21,0
718,C,9.000,1.20650,2000.450,1378.3,0.000000,1725.950,0.0,282.600,0.000,...,39007.600,1.875,1.700,-0.775,0.0,2023,7,3,22,0


(720, 50)


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,observed,seasonal_yr_lag_8760
2023-05-01 00:00:00,4.150,1.28600,NaN,0.0,0.000000,1236.500,0.0,271.025,0.000,0.0,...,3.425,1.450,3.100,0.0,2023,5,1,0,0,0.035054
2023-05-01 01:00:00,4.050,1.28550,1477.100,0.0,0.000000,1220.425,0.0,270.700,0.000,0.0,...,3.325,1.400,3.025,0.0,2023,5,1,1,0,-0.080017
2023-05-01 02:00:00,3.900,1.28375,1477.100,0.0,0.000000,1258.650,0.0,270.200,0.000,0.0,...,3.225,1.225,3.000,0.0,2023,5,1,2,0,-0.103031
2023-05-01 03:00:00,3.750,1.28275,NaN,39890.0,11.549999,1655.175,0.0,269.700,9.300,66888.9,...,3.125,0.950,3.000,0.0,2023,5,1,3,0,-0.114538
2023-05-01 04:00:00,3.700,1.28150,1990.000,564426.7,76.675000,1639.100,0.0,269.450,47.925,412029.7,...,3.025,0.700,2.950,0.0,2023,5,1,4,0,-0.114538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-03 19:00:00,8.400,1.19675,3493.650,1903925.5,84.775000,1967.375,0.0,281.700,32.200,650270.1,...,2.175,1.900,-1.075,0.0,2023,7,3,19,0,0.332963
2023-07-03 20:00:00,8.600,1.20000,3078.750,733045.4,24.550000,1449.500,0.0,282.025,13.875,331501.4,...,2.200,2.000,-0.925,0.0,2023,7,3,20,0,0.252407
2023-07-03 21:00:00,8.875,1.20350,2308.400,147324.3,1.225000,1543.650,0.0,282.350,1.250,108841.0,...,2.250,2.050,-0.950,0.0,2023,7,3,21,0,0.206373
2023-07-03 22:00:00,9.000,1.20650,2000.450,1378.3,0.000000,1725.950,0.0,282.600,0.000,8968.6,...,1.875,1.700,-0.775,0.0,2023,7,3,22,0,0.235140


In [230]:
############ Predictions ################
test_data_a_seasonal_lag.columns

Index(['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'cloud_base_agl:m', 'dew_or_rime:idx', 'dew_point_2m:K',
       'diffuse_rad:W', 'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_density:kgm3', 'snow_depth:cm', 'snow_drift:idx',
       'snow_melt_10min:mm', 'snow_water:kgm2', 'sun_azimuth:d',
       'sun_elevation:d', 'super_cooled_liquid_water:kgm2', 't_1000hPa:K',
       'total_cloud_cover:p', 'visibility:m', 'wind_speed_10m:ms',
       'wind_speed_u_10m:ms', 'wind_speed_v_10

In [305]:
y_pred_a = predict_model(best_model_A, data=test_data_a_seasonal_lag)
df_y_pred_a = y_pred_a[['prediction_label']].copy()
df_y_pred_a.columns = ['pv_measurement']
print(df_y_pred_a) #print the DataFrame

                     pv_measurement
2023-05-01 00:00:00       -1.305224
2023-05-01 01:00:00       -1.305224
2023-05-01 02:00:00       -1.305224
2023-05-01 03:00:00       50.723522
2023-05-01 04:00:00      400.496231
...                             ...
2023-07-03 19:00:00      144.999344
2023-07-03 20:00:00      -37.893918
2023-07-03 21:00:00      -87.719269
2023-07-03 22:00:00      -87.661979
2023-07-03 23:00:00     -123.177398

[720 rows x 1 columns]


In [302]:
y_pred_b = predict_model(best_model_B, data=test_data_b_seasonal_lag)
df_y_pred_b = y_pred_b[['prediction_label']].copy()
df_y_pred_b.columns = ['pv_measurement']
df_y_pred_b #print the DataFrame

,pv_measurement
2023-05-01 00:00:00,0.000000
2023-05-01 01:00:00,0.000000
2023-05-01 02:00:00,0.000000
2023-05-01 03:00:00,4.027863
2023-05-01 04:00:00,50.811223
...,...
2023-07-03 19:00:00,38.730593
2023-07-03 20:00:00,14.568682
2023-07-03 21:00:00,16.768368
2023-07-03 22:00:00,4.355625


In [303]:
y_pred_c = predict_model(best_model_C, data=test_data_c_seasonal_lag)
df_y_pred_c = y_pred_c[['prediction_label']].copy()
df_y_pred_c.columns = ['pv_measurement']
df_y_pred_c #print the DataFrame


,pv_measurement
2023-05-01 00:00:00,0.438456
2023-05-01 01:00:00,0.438456
2023-05-01 02:00:00,0.438456
2023-05-01 03:00:00,2.517790
2023-05-01 04:00:00,13.706025
...,...
2023-07-03 19:00:00,45.463205
2023-07-03 20:00:00,10.738984
2023-07-03 21:00:00,2.196515
2023-07-03 22:00:00,-0.062311


In [306]:
predictions = pd.concat([df_y_pred_a,df_y_pred_b, df_y_pred_c], ignore_index= True)
predictions.tail()

,pv_measurement
2155,45.463205
2156,10.738984
2157,2.196515
2158,-0.062311
2159,-0.453361


In [307]:
predictions_df = predictions.rename(columns={'pv_measurement': 'prediction'})

sample_submission = pd.read_csv('sample_submission.csv')

# Convert all negative predictions to 0
predictions_df.loc[predictions_df['prediction'] < 0, 'prediction'] = 0

# Join the 'id' column from sample_submission with the predictions
sample_submission['prediction'] = predictions_df['prediction']

# Save to CSV
sample_submission.to_csv('pycaret_prediction.csv', index=False)
predictions_df

,prediction
0,0.000000
1,0.000000
2,0.000000
3,50.723522
4,400.496231
...,...
2155,45.463205
2156,10.738984
2157,2.196515
2158,0.000000
